# Distributed haloes cutter
<b>Author</b>: Natalie B. Hogg

**This notebook loads the saved haloes and removes problematic ones, recomputes the shears and saves the surviving sample to a new file**.

## Contents <a name="contents"></a>
1. [Set up](#setup)
2. [Load data](#load)
3. [Compute the shears](#predict_shears)

## Set up <a name="setup"></a>

### Import packages

In [1]:
# computation
import numpy as np
import pandas as pd

# cosmology
from colossus.cosmology import cosmology as colcos
from astropy.table import Table
from astropy.cosmology import FlatLambdaCDM, z_at_value

# monitoring
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

outpath  = r'/home/natalie/Documents/Projects/los_effects/proof_of_concept/figures/distributed_haloes/' 

job_name = 'v2'

### Import `lenstronomy` packages

In [2]:
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.Cosmo.lens_cosmo import LensCosmo

### Start the Universe

In [3]:
cosmology = {'id': 'planck18', 'H0': 67.4, 'Om': 0.315}

colcos.setCosmology(cosmology['id'])

cosmo = FlatLambdaCDM(H0 = cosmology['H0'], Om0 = cosmology['Om']) 

def dA(z1, z2):
    """
    Returns angular diameter distance between two redshifts in Mpc.
    """
    distance = cosmo.angular_diameter_distance_z1z2(z1, z2).value
    return distance

z_observer = 0.0
z_lens     = 0.5 
z_source   = 1.5

d_observer = 0.0
d_od       = dA(z_observer, z_lens)
d_os       = dA(z_observer, z_source)
d_ds       = dA(z_lens, z_source)

## Load the halo data <a name="load"> </a>

In [4]:
haloes_fits = Table.read(outpath + 'total_haloes_dataframe_' + job_name + '.fits')

haloes_dataframe = haloes_fits.to_pandas()

In [5]:
maximum_convergence = 0.5
maximum_Del = 0.1

# split the halo dataframe
surviving_haloes_dataframe = haloes_dataframe.loc[(haloes_dataframe['kappa_ohs'] <= maximum_convergence) & (haloes_dataframe['Del'] <= maximum_Del)]

surviving_halo_number = len(surviving_haloes_dataframe)

# get rid of the kappa ohs and Del
surviving_haloes_dataframe.drop(['kappa_ohs', 'Del'], axis = 1, inplace = True)

## Predict the LOS terms: surviving haloes <a name="predict_shears"></a>

[Back to contents](#contents)

In [6]:
# make a dataframe to save them
surviving_shears_dataframe = pd.DataFrame(columns = ['gamma1_os', 'gamma2_os', 'kappa_os',
                                                     'gamma1_od', 'gamma2_od', 'kappa_od',
                                                     'gamma1_ds', 'gamma2_ds', 'kappa_ds',
                                                     'gamma1_los', 'gamma2_los', 'kappa_los'])

# dummy row so we can write scalars directly to the df
surviving_shears_dataframe = surviving_shears_dataframe.append(pd.Series('dummy'), ignore_index=True)

In [7]:
surviving_halo_number = len(surviving_haloes_dataframe)
surviving_halo_redshift_list = surviving_haloes_dataframe['z'].to_list()
surviving_halo_mass          = surviving_haloes_dataframe['mass'].to_list()
surviving_halo_concentration = surviving_haloes_dataframe['concentration'].to_list()

# OS

In [8]:
Rs_angle = []
alpha_Rs_os = []

for i in tqdm(range(surviving_halo_number)):
    # get the quantities for each halo's redshift
    lens_cosmo = LensCosmo(z_lens=surviving_halo_redshift_list[i],
                           z_source=z_source,
                           cosmo=cosmo)
    
    angles = lens_cosmo.nfw_physical2angle(M=surviving_halo_mass[i],
                                           c=surviving_halo_concentration[i])
    Rs_angle.append(float(angles[0]))
    alpha_Rs_os.append(float(angles[1]))

# add to the dataframe
surviving_haloes_dataframe['Rs'] = Rs_angle # [arcsec]
surviving_haloes_dataframe['alpha_Rs'] = alpha_Rs_os # [arcsec]

  0%|          | 0/27434 [00:00<?, ?it/s]

In [9]:
# get the parameters needed for the lenstronomy kwargs and convert to a list of dictionaries
kwargs_nfw = surviving_haloes_dataframe[['Rs', 'alpha_Rs', 'center_x', 'center_y']].to_dict('records')

single_halo_lens_model = LensModel(lens_model_list=['NFW'], z_source=z_source)

gamma1_ohs = []
gamma2_ohs = []
kappa_ohs  = []

for i in tqdm(range(surviving_halo_number)):
    single_gamma = single_halo_lens_model.gamma(x=0.0, y=0.0, kwargs=[kwargs_nfw[i]])
    gamma1_ohs.append(float(single_gamma[0]))
    gamma2_ohs.append(float(single_gamma[1]))
    
    single_kappa = single_halo_lens_model.kappa(x=0.0, y=0.0, kwargs=[kwargs_nfw[i]])
    kappa_ohs.append(float(single_kappa))

surviving_haloes_dataframe['gamma1_ohs'] = gamma1_ohs
surviving_haloes_dataframe['gamma2_ohs'] = gamma2_ohs 
surviving_haloes_dataframe['kappa_ohs']  = kappa_ohs

kappa_neg = [-k for k in kappa_ohs]

surviving_haloes_dataframe['kappa_neg'] = kappa_neg

  0%|          | 0/27434 [00:00<?, ?it/s]

In [10]:
# Define the (os) setup without the main lens

lens_model_list = ['NFW'] * surviving_halo_number + ['CONVERGENCE'] * surviving_halo_number

lens_redshift_list = 2 * surviving_haloes_dataframe['z'].to_list()

kwargs_neg = [{'kappa': k} for k in surviving_haloes_dataframe['kappa_neg']]

kwargs_lens = kwargs_nfw + kwargs_neg

mp_os_lens_model = LensModel(lens_model_list = lens_model_list, 
                             z_source=z_source,
                             lens_redshift_list=lens_redshift_list,
                             multi_plane=True)

In [11]:
alpha1_os, alpha2_os = mp_os_lens_model.alpha(x=0.0, y=0.0, kwargs=kwargs_lens)

kappa_os = mp_os_lens_model.kappa(x=0.0, y=0.0, kwargs=kwargs_lens)

gamma1_os, gamma2_os = mp_os_lens_model.gamma(x=0.0, y=0.0, kwargs=kwargs_lens)

In [12]:
surviving_shears_dataframe['gamma1_os'] = gamma1_os
surviving_shears_dataframe['gamma2_os'] = gamma2_os
surviving_shears_dataframe['kappa_os']  = kappa_os
surviving_shears_dataframe['alpha1_os'] = alpha1_os
surviving_shears_dataframe['alpha2_os'] = alpha2_os

# OD

In [13]:
# Extract the foreground haloes
foreground_haloes_dataframe = surviving_haloes_dataframe[surviving_haloes_dataframe['z'].between(z_observer, z_lens)]

foreground_halo_number = len(foreground_haloes_dataframe)
foreground_halo_redshift_list = foreground_haloes_dataframe['z'].to_list()
foreground_halo_mass          = foreground_haloes_dataframe['mass'].to_list()
foreground_halo_concentration = foreground_haloes_dataframe['concentration'].to_list()

In [14]:
# Define the new NFW parameters

Rs_angle = []
alpha_Rs_od = []

for i in tqdm(range(foreground_halo_number)):
    # get the quantities for each halo's redshift
    lens_cosmo = LensCosmo(z_lens=foreground_halo_redshift_list[i],
                           z_source=z_lens,
                           cosmo=cosmo)
    
    angles = lens_cosmo.nfw_physical2angle(M=foreground_halo_mass[i],
                                           c=foreground_halo_concentration[i])
    Rs_angle.append(float(angles[0]))
    alpha_Rs_od.append(float(angles[1]))

# add to the dataframe
foreground_haloes_dataframe['Rs'] = Rs_angle # [arcsec]
foreground_haloes_dataframe['alpha_Rs'] = alpha_Rs_od # [arcsec]

  0%|          | 0/10307 [00:00<?, ?it/s]

In [15]:
# Compute the individual convergences and shears

kwargs_nfw = foreground_haloes_dataframe[['Rs','alpha_Rs','center_x','center_y']].to_dict('records')
single_halo_lens_model = LensModel(lens_model_list=['NFW'], z_source=z_lens)

gamma1_ohd = []
gamma2_ohd = []
kappa_ohd  = []

for i in tqdm(range(foreground_halo_number)):
    single_gamma = single_halo_lens_model.gamma(x=0.0, y=0.0, kwargs=[kwargs_nfw[i]])
    gamma1_ohd.append(float(single_gamma[0]))
    gamma2_ohd.append(float(single_gamma[1]))
    
    single_kappa = single_halo_lens_model.kappa(x=0.0, y=0.0, kwargs=[kwargs_nfw[i]])
    kappa_ohd.append(float(single_kappa))

  0%|          | 0/10307 [00:00<?, ?it/s]

In [16]:
# Define the (od) negative kappas from the (os) ones

kappa_neg_os_foreground = foreground_haloes_dataframe['kappa_neg']
kappa_neg_od_foreground = []

for h, kappa in enumerate(kappa_neg_os_foreground):
    z_h = foreground_halo_redshift_list[h]
    d_hd = dA(z_h, z_lens)
    d_hs = dA(z_h, z_source)
    kappa_ohd = d_hd * d_os / d_od / d_os * kappa 
    kappa_neg_od_foreground.append(kappa_ohd)
    
foreground_haloes_dataframe['kappa_neg_od'] = kappa_neg_od_foreground

In [17]:
# Define the (od) setup without the main lens

lens_model_list = ['NFW'] * foreground_halo_number + ['CONVERGENCE'] * foreground_halo_number
lens_redshift_list = 2 * foreground_haloes_dataframe['z'].to_list()
kwargs_neg = [{'kappa': k} for k in foreground_haloes_dataframe['kappa_neg']]
kwargs_lens = kwargs_nfw + kwargs_neg

mp_od_lens_model = LensModel(lens_model_list = lens_model_list, 
                             z_source=z_lens,  # notice we use the main lens redshift as the 'source'
                             lens_redshift_list=lens_redshift_list,
                             multi_plane=True)

In [18]:
alpha1_od, alpha2_od = mp_od_lens_model.alpha(x=0.0, y=0.0, kwargs=kwargs_lens)

kappa_od = mp_od_lens_model.kappa(x=0.0, y=0.0, kwargs=kwargs_lens)

gamma1_od, gamma2_od = mp_od_lens_model.gamma(x=0.0, y=0.0, kwargs=kwargs_lens)

In [19]:
# save in dataframe
surviving_shears_dataframe['alpha1_od'] = alpha1_od
surviving_shears_dataframe['alpha2_od'] = alpha2_od
surviving_shears_dataframe['gamma1_od'] = gamma1_od
surviving_shears_dataframe['gamma2_od'] = gamma2_od
surviving_shears_dataframe['kappa_od']  = kappa_od

# DS

In [20]:
# Extract the background haloes
background_haloes_dataframe = surviving_haloes_dataframe[surviving_haloes_dataframe['z'].between(z_lens, z_source)]

background_halo_number = len(background_haloes_dataframe)
background_halo_redshift_list = background_haloes_dataframe['z'].to_list()
background_halo_mass          = background_haloes_dataframe['mass'].to_list()
background_halo_concentration = background_haloes_dataframe['concentration'].to_list()

In [21]:
kappa_dhs  = []
gamma1_dhs = []
gamma2_dhs = []

for h in tqdm(range(background_halo_number)):
    
    z_h  = background_halo_redshift_list[h]
    d_dh = dA(z_lens, z_h)
    d_oh = dA(z_observer, z_h)
    
    kap_ohs  = kappa_ohs[h]
    gam1_ohs = gamma1_ohs[h]
    gam2_ohs = gamma2_ohs[h]
    
    kappa_dhs.append(d_dh * d_os / d_ds / d_oh * kap_ohs)
    gamma1_dhs.append(d_dh * d_os / d_ds / d_oh * gam1_ohs)
    gamma2_dhs.append(d_dh * d_os / d_ds / d_oh * gam2_ohs)
    
background_haloes_dataframe['kappa_dhs'] = kappa_dhs
background_haloes_dataframe['gamma1_dhs'] = gamma1_dhs
background_haloes_dataframe['gamma2_dhs'] = gamma2_dhs

  0%|          | 0/17127 [00:00<?, ?it/s]

In [22]:
kappa_ds  = 0 
gamma1_ds = sum(gamma1_dhs)
gamma2_ds = sum(gamma2_dhs)

surviving_shears_dataframe['gamma1_ds'] = gamma1_ds
surviving_shears_dataframe['gamma2_ds'] = gamma2_ds
surviving_shears_dataframe['kappa_ds']  = kappa_ds

### Get the LOS component

In [23]:
gamma1_LOS = gamma1_os + gamma1_od - gamma1_ds
gamma2_LOS = gamma2_os + gamma2_od - gamma2_ds
kappa_LOS  = kappa_os  + kappa_od  - kappa_ds

surviving_shears_dataframe['gamma1_los'] = gamma1_LOS
surviving_shears_dataframe['gamma2_los'] = gamma2_LOS
surviving_shears_dataframe['kappa_los']  = kappa_LOS

In [24]:
print('\nThe predicted shear components with problematic haloes are:')
print('gamma_os  = ({:.3e}, {:.3e})'.format(gamma1_os, gamma2_os))
print('gamma_od  = ({:.3e}, {:.3e})'.format(gamma1_od, gamma2_od))
print('gamma_ds  = ({:.3e}, {:.3e})'.format(gamma1_ds, gamma2_ds))
print('gamma_LOS = ({:.3e}, {:.3e})'.format(gamma1_LOS, gamma2_LOS))


The predicted shear components with problematic haloes are:
gamma_os  = (2.832e-03, 1.785e-02)
gamma_od  = (3.810e-04, 1.125e-02)
gamma_ds  = (2.022e-03, -6.531e-03)
gamma_LOS = (1.190e-03, 3.563e-02)


In [25]:
print('\nThe predicted convergence components are:')
print('kappa_os  = {:.3e}.'.format(kappa_os))
print('kappa_od  = {:.3e}.'.format(kappa_od))
print('kappa_ds  = {:.3e}.'.format(kappa_ds))
print('kappa_LOS = {:.3e}.'.format(kappa_LOS))


The predicted convergence components are:
kappa_os  = -4.093e-04.
kappa_od  = -3.691e-02.
kappa_ds  = 0.000e+00.
kappa_LOS = -3.732e-02.


# Save the final dataframes to file

In [26]:
# write haloes to fits
surviving_haloes_fits = Table.from_pandas(surviving_haloes_dataframe)
surviving_haloes_fits.write(outpath + 'surviving_haloes_dataframe_' + job_name + '.fits', overwrite = True)

# write shears to csv
surviving_shears_dataframe.to_csv(outpath + 'surviving_shears_dataframe_' + job_name + '.csv', index = False)

[Back to contents](#contents)